# DATASCI 503, Group Work 10: Stochastic Gradient Descent for Loss Functions

**Instructions:** During lab section, and afterward as necessary, you will collaborate in two-person teams to complete the problems that are interspersed below. The GSI will help individual teams encountering difficulty, make announcements addressing common issues, and help ensure progress for all teams. **During lab, feel free to flag down your GSI to ask questions at any point!**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

## Part 1: Gradient Descent for a Given Function

---

**Problem 1:** Gradient Descent for a Function of One Argument

Recall that gradient descent is an optimization algorithm used to minimize some function by iteratively moving in the direction of steepest descent as defined by the negative of the function's gradient.

Namely, the procedure is as follows:
1. Start with a random point $w$.
2. Update $w$ by moving in the direction of the negative gradient of the function at $w$: $$w \leftarrow w - \eta \nabla f(w).$$
Here, $\eta$ is the *learning rate*; it controls the size of the step we're taking in the direction of the negative gradient.
3. Repeat step 2 until the variable $w$ changes very little between iterations.

Below, write code to minimize $$f(w) = (w -2)^2$$ using gradient descent. Your function should return the minimizer $w^*$.

In [ ]:
def gradient_descent_1d(epsilon: float, initial_w: float, eta: float) -> float:
    """
    Minimize f(w) = (w - 2)^2 using gradient descent.

    Args:
        epsilon: tolerance for convergence
        initial_w: initial value of w
        eta: learning rate

    Returns:
        The minimizer w* (approximately 2.0)
    """
    # BEGIN SOLUTION
    # The gradient of f(w) = (w-2)^2 is f'(w) = 2*(w-2)
    w_prev = float("inf")
    weight = initial_w

    while abs(weight - w_prev) > epsilon:
        w_prev = weight
        gradient = 2 * (weight - 2)
        weight = weight - eta * gradient

    return weight
    # END SOLUTION

In [ ]:
# Test assertions
result = gradient_descent_1d(epsilon=0.0001, initial_w=5.0, eta=0.1)
assert abs(result - 2) < 0.001, f"Expected result near 2, got {result}"

result_neg = gradient_descent_1d(epsilon=0.0001, initial_w=-3.0, eta=0.1)
assert abs(result_neg - 2) < 0.001, f"Expected result near 2 from negative start, got {result_neg}"

print("All tests passed!")

# BEGIN HIDDEN TESTS
# Test with different learning rates
result_slow = gradient_descent_1d(epsilon=0.0001, initial_w=10.0, eta=0.05)
assert abs(result_slow - 2) < 0.001, f"Failed with slower learning rate: {result_slow}"

result_tight = gradient_descent_1d(epsilon=0.00001, initial_w=0.0, eta=0.2)
assert abs(result_tight - 2) < 0.0001, f"Failed with tighter epsilon: {result_tight}"
# END HIDDEN TESTS

---

**Problem 2:** Gradient Descent for a Function of Two Arguments

Now we want to minimize the Rosenbrock function:

$$f(w_1, w_2) = (1-w_1)^2 + 100 (w_2 - w_1^2)^2$$

using gradient descent. The minimum is at $(1, 1)$.

**Hint:** You can treat $w$ as a vector, but this is not necessary.

**Hint:** If your algorithm is not converging, try adjusting the learning rate first. The Rosenbrock function has a narrow curved valley, so a smaller learning rate may be needed.

In [ ]:
def gradient_descent_2d(epsilon: float, initial_w: np.ndarray, eta: float) -> np.ndarray:
    """
    Minimize the Rosenbrock function using gradient descent.

    Args:
        epsilon: tolerance for convergence
        initial_w: initial weight vector [w1, w2]
        eta: learning rate

    Returns:
        The minimizer (approximately [1.0, 1.0])
    """
    # BEGIN SOLUTION
    # Gradient: df/dw1 = -2(1-w1) + 100*2*(w2-w1^2)*(-2*w1)
    #           df/dw2 = 100*2*(w2-w1^2)
    w_prev = np.array([float("inf"), float("inf")])
    weights = initial_w.copy()

    while np.linalg.norm(weights - w_prev, 2) > epsilon:
        w1, w2 = weights
        grad_w1 = -2 * (1 - w1) + 400 * (w2 - w1**2) * (-w1)
        grad_w2 = 200 * (w2 - w1**2)
        gradient = np.array([grad_w1, grad_w2])

        w_prev = weights.copy()
        weights = weights - eta * gradient

    return weights
    # END SOLUTION

In [ ]:
# Test assertions
result = gradient_descent_2d(epsilon=0.00001, initial_w=np.array([2.0, 1.0]), eta=0.001)
expected = np.array([1.0, 1.0])
assert np.linalg.norm(result - expected, 2) < 0.05, f"Expected near [1, 1], got {result}"

result2 = gradient_descent_2d(epsilon=0.00001, initial_w=np.array([0.0, 0.0]), eta=0.001)
assert (
    np.linalg.norm(result2 - expected, 2) < 0.05
), f"Expected near [1, 1] from origin, got {result2}"

print("All tests passed!")

# BEGIN HIDDEN TESTS
result3 = gradient_descent_2d(epsilon=0.000001, initial_w=np.array([1.5, 2.0]), eta=0.001)
assert np.linalg.norm(result3 - expected, 2) < 0.02, f"Failed with different start: {result3}"

result4 = gradient_descent_2d(epsilon=0.00001, initial_w=np.array([-0.5, 0.5]), eta=0.001)
assert np.linalg.norm(result4 - expected, 2) < 0.05, f"Failed with negative start: {result4}"
# END HIDDEN TESTS

## Part 2: Gradient Descent for Linear Regression

The previous functions just depended on some parameters $(w_1, w_2)$. Now we are interested in optimizing loss functions that depend on data $(X, y)$.

---

**Problem 3:** Simple Linear Regression

We are interested in:

1. Fitting a simple linear model:
$$\widehat{y}_i = \alpha + \beta x_i$$

2. By minimizing the mean squared error loss:
$$L(\alpha, \beta) = \frac{1}{2n}\sum_{i=1}^n (y_i - \hat{y}_i)^2 = \frac{1}{2n}\sum_{i=1}^n (y_i - \alpha - \beta x_i)^2$$

   (Note: The $\frac{1}{2}$ factor is a convention in machine learning; it simplifies the gradient and does not change the minimizer.)

3. Using gradient descent updates:
$$\alpha_k = \alpha_{k-1} - \eta \frac{\partial L}{\partial \alpha}$$
$$\beta_k = \beta_{k-1} - \eta \frac{\partial L}{\partial \beta}$$

You can check convergence of gradient descent in either of two ways:

- Check if the parameter changes are smaller than epsilon:
$$|\alpha_k - \alpha_{k-1}| + |\beta_k - \beta_{k-1}| < \epsilon$$

- Or check whether the change in the loss function is smaller than epsilon:
$$|L_k - L_{k-1}| < \epsilon$$

In [ ]:
def simple_linear_regression_gd(
    features: np.ndarray,
    targets: np.ndarray,
    eta: float,
    initial_alpha: float,
    initial_beta: float,
    epsilon: float,
) -> tuple[float, float]:
    """
    Fit simple linear regression using gradient descent.

    Args:
        features: input feature array (1D)
        targets: target values array (1D)
        eta: learning rate
        initial_alpha: initial intercept value
        initial_beta: initial slope value
        epsilon: convergence tolerance

    Returns:
        Tuple of (alpha, beta) - the fitted intercept and slope
    """
    # BEGIN SOLUTION
    # Gradients: dL/dalpha = -mean(y - alpha - beta*x)
    #            dL/dbeta = -mean((y - alpha - beta*x) * x)

    def compute_loss(alpha, beta):
        residuals = targets - alpha - beta * features
        return 0.5 * np.mean(residuals**2)

    def grad_alpha(alpha, beta):
        return -np.mean(targets - alpha - beta * features)

    def grad_beta(alpha, beta):
        return -np.mean((targets - alpha - beta * features) * features)

    alpha_prev, beta_prev = float("inf"), float("inf")
    alpha, beta = initial_alpha, initial_beta
    loss_prev = float("inf")

    while True:
        loss_current = compute_loss(alpha, beta)
        param_change = abs(alpha - alpha_prev) + abs(beta - beta_prev)
        loss_change = abs(loss_current - loss_prev)

        if param_change < epsilon or loss_change < epsilon:
            break

        alpha_prev, beta_prev = alpha, beta
        loss_prev = loss_current

        # Compute both gradients at the current point before updating
        grad_a = grad_alpha(alpha, beta)
        grad_b = grad_beta(alpha, beta)
        alpha = alpha - eta * grad_a
        beta = beta - eta * grad_b

    return alpha, beta
    # END SOLUTION

Use the following cells to test your solution. The obtained coefficients do not need to be exactly the same as scikit-learn's, but should be close. Making epsilon smaller will yield more accurate results.

In [ ]:
# Generate some random data for testing
np.random.seed(42)
x_test = np.random.rand(100)
true_alpha = np.random.rand(1)
true_beta = np.random.rand(1)
y_test = true_alpha + x_test * true_beta + np.random.rand(100) / 10

Check the solution using scikit-learn:

In [ ]:
from sklearn.linear_model import LinearRegression

lr_simple = LinearRegression()
_ = lr_simple.fit(x_test[:, None], y_test)
print(f"Sklearn intercept: {lr_simple.intercept_}, slope: {lr_simple.coef_[0]}")

In [ ]:
# Test assertions
eta, initial_alpha, initial_beta, epsilon = 0.1, 0, 0, 0.0001
alpha_hat, beta_hat = simple_linear_regression_gd(
    x_test, y_test, eta, initial_alpha, initial_beta, epsilon
)
print(f"Gradient descent intercept: {alpha_hat}, slope: {beta_hat}")

sklearn_params = np.array([lr_simple.intercept_, lr_simple.coef_[0]])
gd_params = np.array([alpha_hat, beta_hat])
assert (
    np.linalg.norm(gd_params - sklearn_params, 1) < 0.1
), f"Parameters differ too much from sklearn: {gd_params} vs {sklearn_params}"

print("All tests passed!")

# BEGIN HIDDEN TESTS
# Test with different initial values
alpha_hat2, beta_hat2 = simple_linear_regression_gd(
    x_test, y_test, eta=0.2, initial_alpha=1.0, initial_beta=1.0, epsilon=0.00001
)
assert (
    np.linalg.norm(np.array([alpha_hat2, beta_hat2]) - sklearn_params, 1) < 0.05
), "Failed with different initial values"

# Test with smaller learning rate
alpha_hat3, beta_hat3 = simple_linear_regression_gd(
    x_test, y_test, eta=0.05, initial_alpha=0.5, initial_beta=0.5, epsilon=0.0001
)
assert (
    np.linalg.norm(np.array([alpha_hat3, beta_hat3]) - sklearn_params, 1) < 0.1
), "Failed with smaller learning rate"
# END HIDDEN TESTS

---

**Problem 4:** Multivariate Linear Regression

Now implement gradient descent for multivariate linear regression with multiple features.

The loss function for multivariate linear regression is:

$$L(w) = \frac{1}{2n}\sum_{i=1}^n (y_i - x_i^T w)^2 = \frac{1}{2n}||Y - X w||^2$$

where each $x_i$ is a row vector (or correspondingly, $X$ is a matrix with rows as samples), and $y_i$ is the corresponding element of the target vector $Y$.

The gradient of this loss function is:

$$\nabla L = \frac{1}{n} X^T(Xw - Y)$$

The gradient descent update rule in vector form is:

$$w_k = w_{k-1} - \eta \nabla L$$

where $w$ is the weight vector, $\eta$ is the learning rate, and $\nabla L$ is the gradient.

Gradient descent converges when either:
- The weight change is smaller than epsilon: $||w_k - w_{k-1}|| < \epsilon$
- The loss change is smaller than epsilon: $|L_k - L_{k-1}| < \epsilon$

Unlike in the previous question, please also keep track of the loss at each iteration and return the complete loss history.

**Note:** Your function should automatically add a column of ones to X to fit the intercept term.

In [ ]:
def multivariate_linear_regression_gd(
    features: np.ndarray,
    targets: np.ndarray,
    eta: float,
    initial_weights: np.ndarray,
    epsilon: float,
) -> tuple[np.ndarray, list]:
    """
    Fit multivariate linear regression using gradient descent.

    Args:
        features: input feature matrix (n_samples x n_features)
        targets: target values array (n_samples,)
        eta: learning rate
        initial_weights: initial weight vector (n_features + 1,) including intercept
        epsilon: convergence tolerance

    Returns:
        Tuple of (weights, losses) where weights is the fitted weight vector
        and losses is the list of loss values at each iteration
    """
    # BEGIN SOLUTION
    num_samples = features.shape[0]

    # Add column of ones for intercept
    ones_column = np.ones((num_samples, 1))
    design_matrix = np.hstack((ones_column, features))
    target_vector = np.array(targets).reshape((num_samples, 1))

    def compute_loss(weights):
        residuals = target_vector - design_matrix @ weights
        return 0.5 * np.mean(residuals**2)

    def compute_gradient(weights):
        return design_matrix.T @ (design_matrix @ weights - target_vector) / num_samples

    losses = []
    weights = initial_weights.copy()
    weights_prev = np.ones_like(weights) * 100.0
    loss_prev = float("inf")

    while True:
        loss_current = compute_loss(weights)
        weight_change = np.linalg.norm(weights - weights_prev, 1)
        loss_change = abs(loss_current - loss_prev)

        if weight_change < epsilon or loss_change < epsilon:
            break

        weights_prev = weights.copy()
        loss_prev = loss_current
        losses.append(loss_current)

        gradient = compute_gradient(weights)
        weights = weights - eta * gradient

    return weights, losses
    # END SOLUTION

We will use real-world data to test our solution. The coefficients obtained may not match sklearn exactly, but the training loss should decrease monotonically and be comparable to sklearn's solution.

In [ ]:
# Load and prepare the California housing dataset
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing_data = fetch_california_housing(return_X_y=True, as_frame=True)
housing_features = housing_data[0]
housing_targets = housing_data[1]
housing_train_features, housing_val_features, housing_train_targets, housing_val_targets = (
    train_test_split(housing_features, housing_targets, test_size=0.3, random_state=42)
)
housing_scaler = StandardScaler()
housing_train_scaled = housing_scaler.fit_transform(housing_train_features)

In [ ]:
# Test assertions
np.random.seed(42)
housing_num_weights = housing_features.shape[1] + 1
housing_weights_gd, housing_loss_history = multivariate_linear_regression_gd(
    housing_train_scaled,
    housing_train_targets,
    eta=0.01,
    initial_weights=np.random.randn(housing_num_weights, 1),
    epsilon=0.001,
)

# Compute predictions on validation set
housing_val_scaled = housing_scaler.transform(housing_val_features)
housing_val_with_intercept = np.hstack(
    (np.ones((housing_val_features.shape[0], 1)), housing_val_scaled)
)
housing_predictions_val = housing_val_with_intercept @ housing_weights_gd
housing_val_targets_reshaped = np.array(housing_val_targets).reshape(
    (housing_val_targets.shape[0], 1)
)

plt.plot(np.arange(1, len(housing_loss_history) + 1), housing_loss_history, color="red")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.title("Training Loss over Iterations")
plt.show()

print(f"Final training loss: {housing_loss_history[-1]}")
val_loss = 0.5 * np.mean((housing_val_targets_reshaped - housing_predictions_val) ** 2)
print(f"Validation loss: {val_loss}")

# Verify loss decreases (allowing small numerical fluctuations)
assert len(housing_loss_history) > 1, "Should have multiple iterations"
assert housing_loss_history[-1] < housing_loss_history[0], "Loss should decrease overall"

print("All tests passed!")

# BEGIN HIDDEN TESTS
# Check loss is decreasing monotonically (with small tolerance for numerical issues)
for idx in range(1, len(housing_loss_history)):
    assert (
        housing_loss_history[idx] <= housing_loss_history[idx - 1] + 1e-6
    ), f"Loss not monotonically decreasing at {idx}"

# Check final loss is reasonable (should be close to sklearn's ~0.26)
assert housing_loss_history[-1] < 0.5, f"Final loss too high: {housing_loss_history[-1]}"
# END HIDDEN TESTS

In [ ]:
# Compare with sklearn's LinearRegression
housing_sklearn_model = LinearRegression(fit_intercept=True)
_ = housing_sklearn_model.fit(housing_train_scaled, housing_train_targets)
print(f"Sklearn intercept: {housing_sklearn_model.intercept_}")
print(f"Sklearn coefficients: {housing_sklearn_model.coef_}")

In [ ]:
# Compare losses between gradient descent and sklearn
sklearn_train_pred = housing_sklearn_model.predict(housing_train_scaled)
housing_sklearn_train_loss = 0.5 * np.mean((housing_train_targets - sklearn_train_pred) ** 2)
sklearn_val_pred = housing_sklearn_model.predict(housing_val_scaled)
housing_sklearn_val_loss = 0.5 * np.mean((housing_val_targets - sklearn_val_pred) ** 2)
print(f"Sklearn training loss: {housing_sklearn_train_loss}")
print(f"Sklearn validation loss: {housing_sklearn_val_loss}")

---

**Problem 5:** Minibatch Gradient Descent

In mini-batch gradient descent, we apply gradient descent to a subset of the data (called a mini-batch or batch) at each iteration. The algorithm is as follows:

**Inputs:** $X, Y, w_0, \eta, \epsilon, |B|$ (batch size)

**Step 0:** Set $n$ = number of samples and calculate the number of batches as $m = \lceil n/|B| \rceil$

**Step 1:** Initialize weights $w_k = w_0$

**Step 2:** Repeat until $||w_k - w_{k-1}|| < \epsilon$:

- **Step 2a:** Randomly shuffle the data and split into $m$ batches: $B_1, B_2, \ldots, B_m$

- **Step 2b:** For each batch $j \in \{1, 2, \ldots, m\}$, apply the gradient descent update using only that batch's data:
$$w_k = w_{k-1} - \eta \frac{1}{|B_j|} X_{B_j}^T(X_{B_j} w_{k-1} - Y_{B_j})$$

- **Step 2c:** After processing all batches, compute and store the loss on the **full** dataset:
$$L_k = \frac{1}{2n}||Y - X w_k||^2$$

**Output:** $w_k$, $[L_1, L_2, \ldots, L_k]$

Write a function for mini-batch gradient descent that returns $\widehat{w}$ and the loss history. This function should work with any batch size.

**Hint:** You can use `np.random.permutation(n)` to get a random permutation of indices, then use these to shuffle both X and y together:

```python
# Shuffle X and y together using the same permutation
shuffle_indices = np.random.permutation(num_samples)
X_shuffled = X[shuffle_indices]
y_shuffled = y[shuffle_indices]

# Get the j-th minibatch
X_batch = X_shuffled[j * batch_size : (j + 1) * batch_size]
y_batch = y_shuffled[j * batch_size : (j + 1) * batch_size]
```

Note that $X$ and $y$ must **always be shuffled together** (think about why this is necessary!).

Use a for loop with index $j$ going from 0 to $\lceil n / |B| \rceil - 1$ and apply a gradient descent step on the data from the $j$-th batch. If $|B|$ does not divide $n$ evenly, the last batch will be smaller.

In [ ]:
def minibatch_gradient_descent(
    features: np.ndarray,
    targets: np.ndarray,
    eta: float,
    initial_weights: np.ndarray,
    epsilon: float,
    batch_size: int,
) -> tuple[np.ndarray, list]:
    """
    Fit multivariate linear regression using mini-batch gradient descent.

    Args:
        features: input feature matrix (n_samples x n_features)
        targets: target values array (n_samples,)
        eta: learning rate
        initial_weights: initial weight vector (n_features + 1,) including intercept
        epsilon: convergence tolerance
        batch_size: size of each mini-batch

    Returns:
        Tuple of (weights, losses) where weights is the fitted weight vector
        and losses is the list of loss values at the end of each epoch
    """
    # BEGIN SOLUTION
    num_samples = features.shape[0]

    # Add column of ones for intercept
    ones_column = np.ones((num_samples, 1))
    design_matrix = np.hstack((ones_column, features))
    target_vector = np.array(targets).reshape((num_samples, 1))

    num_batches = (num_samples + batch_size - 1) // batch_size  # Ceiling division

    def compute_loss(weights):
        residuals = target_vector - design_matrix @ weights
        return 0.5 * np.mean(residuals**2)

    def compute_batch_gradient(x_batch, y_batch, weights, current_batch_size):
        return x_batch.T @ (x_batch @ weights - y_batch) / current_batch_size

    losses = []
    weights = initial_weights.copy()
    weights_prev = np.ones_like(weights) * 100.0

    while np.linalg.norm(weights - weights_prev, 1) > epsilon:
        weights_prev = weights.copy()

        # Shuffle data at the start of each epoch
        shuffle_indices = np.random.permutation(num_samples)
        design_shuffled = design_matrix[shuffle_indices]
        target_shuffled = target_vector[shuffle_indices]

        # Process each batch
        for batch_idx in range(num_batches):
            start_idx = batch_idx * batch_size
            end_idx = min((batch_idx + 1) * batch_size, num_samples)

            x_batch = design_shuffled[start_idx:end_idx]
            y_batch = target_shuffled[start_idx:end_idx]
            current_batch_size = end_idx - start_idx

            gradient = compute_batch_gradient(x_batch, y_batch, weights, current_batch_size)
            weights = weights - eta * gradient

        # Record loss after each epoch
        losses.append(compute_loss(weights))

    return weights, losses
    # END SOLUTION

We can reuse the test setup from the previous question, as we are again solving multivariate linear regression, this time with mini-batch optimization. The same caveats apply.

In [ ]:
# Test assertions
np.random.seed(42)
housing_weights_minibatch, housing_loss_history_minibatch = minibatch_gradient_descent(
    housing_train_scaled,
    housing_train_targets,
    eta=0.01,
    initial_weights=np.random.randn(housing_num_weights, 1),
    epsilon=0.001,
    batch_size=512,
)

# Compute predictions on validation set
housing_predictions_minibatch = housing_val_with_intercept @ housing_weights_minibatch

plt.plot(
    np.arange(1, len(housing_loss_history_minibatch) + 1),
    housing_loss_history_minibatch,
    color="red",
)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss over Epochs (Mini-batch GD)")
plt.show()

print(f"Final training loss: {housing_loss_history_minibatch[-1]}")
minibatch_val_loss = 0.5 * np.mean(
    (housing_val_targets_reshaped - housing_predictions_minibatch) ** 2
)
print(f"Validation loss: {minibatch_val_loss}")

# Verify loss decreases overall
assert len(housing_loss_history_minibatch) > 1, "Should have multiple epochs"
assert (
    housing_loss_history_minibatch[-1] < housing_loss_history_minibatch[0]
), "Loss should decrease overall"

print("All tests passed!")

# BEGIN HIDDEN TESTS
# Check final loss is reasonable (should converge to similar value as full-batch)
assert (
    housing_loss_history_minibatch[-1] < 0.5
), f"Final loss too high: {housing_loss_history_minibatch[-1]}"

# Test with different batch size
np.random.seed(123)
housing_weights_small_batch, housing_losses_small_batch = minibatch_gradient_descent(
    housing_train_scaled,
    housing_train_targets,
    eta=0.01,
    initial_weights=np.random.randn(housing_num_weights, 1),
    epsilon=0.001,
    batch_size=128,
)
assert (
    housing_losses_small_batch[-1] < 0.5
), f"Failed with smaller batch size: {housing_losses_small_batch[-1]}"
# END HIDDEN TESTS

In [ ]:
# Compare with sklearn's LinearRegression (same as before)
print(f"Sklearn intercept: {housing_sklearn_model.intercept_}")
print(f"Sklearn coefficients: {housing_sklearn_model.coef_}")

In [ ]:
# Compare losses
print(f"Sklearn training loss: {housing_sklearn_train_loss}")
print(f"Sklearn validation loss: {housing_sklearn_val_loss}")